## Train LSTM on TBI signal data

Code to train an LSTM on the TBI data to predict hypoxemia in the future (low SAO2).

Note that the data is private and we are unable to make it publicly available in this repo.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,4"

import numpy as np
from tbi_downstream_prediction import split_data

import keras
from keras.utils import multi_gpu_model
from keras.layers import Input, LSTM, Dense, Dropout
from keras.models import Sequential, load_model, Model
from matplotlib import cm, pyplot as plt
from sklearn import metrics
from os.path import expanduser as eu
from os.path import isfile, join
import numpy as np
import random, time

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(allow_soft_placement=True,gpu_options = tf.GPUOptions(allow_growth=True))
set_session(tf.Session(config=config))
GPUNUM = len(os.environ["CUDA_VISIBLE_DEVICES"].split(","))

PATH = "/homes/gws/hughchen/phase/tbi_subset/"
DPATH = PATH+"tbi/processed_data/hypoxemia/"
data_type = "raw[top11]"

feat_lst = ["ECGRATE", "ETCO2", "ETSEV", "ETSEVO", "FIO2", "NIBPD", "NIBPM", 
            "NIBPS","PEAK", "PEEP", "PIP", "RESPRATE", "SAO2", "TEMP1", "TV"]

# Exclude these features
exclude_feat_lst = ["ETSEV", "PIP", "PEEP", "TV"]
feat_inds = np.array([feat_lst.index(feat) for feat in feat_lst if feat not in exclude_feat_lst])
feat_lst2 = [feat for feat in feat_lst if feat not in exclude_feat_lst]

y_tbi = np.load(DPATH+"tbiy.npy")
X_tbi = np.load(DPATH+"X_tbi_imp_standard.npy")

X_tbi2 = X_tbi[:,feat_inds,:]
(X_test, y_test, X_valid, y_valid, X_train, y_train) = split_data(DPATH,X_tbi2,y_tbi,flatten=False)

PATH = "/homes/gws/hughchen/phase/tbi_subset/"
RESULTPATH = PATH+"results/"
label_type = "desat_bool92_5_nodesat"
lstm_type = "biglstmdropoutv3_{}".format(label_type)
RESDIR = '{}{}/'.format(RESULTPATH, lstm_type)
if not os.path.exists(RESDIR): os.makedirs(RESDIR)

### Hyperparameter tuning

In [ ]:
# Set of hyperparameters to tune over
node_lst   = [50, 100, 200]
opt_lst    = ["rmsprop", "adam", "sgd"]
lr_lst     = [0.01, 0.001, 0.0001]
drop_lst   = [0.3, 0.5, 0.7]
ISTUNE = True

# Greedily find best opt
min_loss_lst = [
    create_train_model(opt_name="rmsprop",lr=0.001,drop=0.5,
                       b_size=1000,nodesize=para)
    for para in node_lst
]
best_nodesize = node_lst[min_loss_lst.index(min(min_loss_lst))]
print("best best_nodesize: {}".format(best_nodesize))

# Greedily find best opt
min_loss_lst = [
    create_train_model(opt_name=para,lr=0.001,drop=0.5,
                       b_size=1000,nodesize=best_nodesize)
    for para in opt_lst
]
best_optname = opt_lst[min_loss_lst.index(min(min_loss_lst))]
print("best optname: {}".format(best_optname))

# Greedily find best lr
min_loss_lst = [
    create_train_model(opt_name=best_optname,lr=para,drop=0.5,
                       b_size=1000,nodesize=best_nodesize)
    for para in lr_lst
]
best_lr = lr_lst[min_loss_lst.index(min(min_loss_lst))]
print("best lr: {}".format(best_lr))

# Greedily find best dropout
min_loss_lst = [
    create_train_model(opt_name=best_optname,lr=best_lr,drop=para,
                       b_size=1000,nodesize=best_nodesize)
    for para in drop_lst
]
best_drop = drop_lst[min_loss_lst.index(min(min_loss_lst))]
print("best drop: {}".format(best_drop))

### Train the model with the best hyperparameters

In [ ]:
create_train_model(opt_name=best_optname,lr=best_lr,drop=best_drop,
                   b_size=1000,nodesize=best_nodesize,epoch_num=200,
                   is_tune=False)
eval_test_model(opt_name=best_optname,lr=best_lr,drop=best_drop,
                b_size=1000,nodesize=best_nodesize,epoch_num=200)